Burgers equation in time - Generating training data

In [ ]:
import numpy as np
from IPython.display import clear_output
from scipy import interpolate

In [ ]:
Nx = 2**6
Nt = 2**18
dx = 1/Nx
dz = 4*dx
dt = 1/Nt
xx = np.arange(0,1+dx,dx)
zz = np.arange(0,1+dz,dz)
tt = np.arange(0,0.25+dt,dt)
nu = 0.025
resolution = 2048*2

def compute_u0(xx,zz, mean=0, std_dev=0):
    mean = 0
    std_dev = 1
    u0_ = np.random.normal(mean, std_dev, zz.shape[0])
    spl = interpolate.splrep(zz,u0_)
    u0 = interpolate.splev(xx,spl)
    u0[0] = 0
    u0[-1] = 0
    return u0

input_list = []
output_list = []
N_samples = 100
for n in range(N_samples):
    u0 = compute_u0(xx,zz)
    uh = np.zeros((xx.shape[0],tt.shape[0]))
    uh[:,0] = u0
    last_j=0
    for j in range(0, tt.shape[0]-1):
        for i in range(1, xx.shape[0]-1):
            uh[i,j+1] = uh[i,j] + nu*dt*(uh[i+1,j] - 2*uh[i,j] + uh[i-1,j])/(dx**2) - 0.5*dt*(uh[i,j]**2-uh[i-1,j]**2)/dx
        if np.mod(j,resolution)==0 and not(j==0):
            factor = np.amax(np.abs(uh[:,last_j]))
            input_list.append(1/factor*uh[:,last_j])
            output_list.append(1/factor*(uh[:,j]))
            last_j=j
    clear_output(wait=True)
    print('Advancing: '+str((n+1)/N_samples*100) +'%')

input = np.array(input_list)
output = np.array(output_list)

from sklearn.model_selection import train_test_split
input_train, input_test, output_train, output_test = train_test_split(input, output, test_size=0.2)

np.save('../data/Burgers_time/input_train.npy', input_train)
np.save('../data/Burgers_time/output_train.npy', output_train)
np.save('../data/Burgers_time/input_test.npy', input_test)
np.save('../data/Burgers_time/output_test.npy', output_test)